# Обработка признаков

В этом домашнем задании вы будете решать задачу предсказания стоимости автомобилей по их различным характеристикам.

In [539]:
import pandas as pd

RANDOM_STATE = 42

In [540]:
df = pd.read_csv("https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/cars_prices.csv", decimal='.')

In [541]:
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


### Описание некоторых признаков

`symboling` - rating corresponds to the degree to which the auto is more risky than its price indicates (+3 more risk and -3 is pretty safe)  
`make` - car types (i.e. car brand)  
`fuel-type` - types of fuel (gas or diesel)  
`aspiration` - engine aspiration (standard or turbo)  
`num-of-doors` - numbers of doors (two or four)  
`body-style` - car body style (sedan or hachback)  
`drive-wheels` - which types of drive wheel (forward-fwd, reversed-rwd)  
`engine-location` - engine mounted location (front or back)  
`wheel-base` - расстояние между осями передних и задних колес  
`length` - car lenght  
`weight` - car weight  
`width` - car width  
`height` - car height  

In [542]:
df.shape

(205, 26)

In [543]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized-losses  205 non-null    object 
 2   make               205 non-null    object 
 3   fuel-type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num-of-doors       205 non-null    object 
 6   body-style         205 non-null    object 
 7   drive-wheels       205 non-null    object 
 8   engine-location    205 non-null    object 
 9   wheel-base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb-weight        205 non-null    int64  
 14  engine-type        205 non-null    object 
 15  num-of-cylinders   205 non-null    object 
 16  engine-size        205 non

## Заполнение пропусков

Пропуски в этом датасете обозначены как `?`

In [544]:
for c in df.columns:
    print(c, len(df[df[c] == '?']))

symboling 0
normalized-losses 41
make 0
fuel-type 0
aspiration 0
num-of-doors 2
body-style 0
drive-wheels 0
engine-location 0
wheel-base 0
length 0
width 0
height 0
curb-weight 0
engine-type 0
num-of-cylinders 0
engine-size 0
fuel-system 0
bore 4
stroke 4
compression-ratio 0
horsepower 2
peak-rpm 2
city-mpg 0
highway-mpg 0
price 4


Удалите строки, для которых неизвестно значение price, так как это целевая переменная.

## Вопрос для Quiz

Сколько строк осталось в данных?

In [545]:
df = df[df['price'] != '?']
print(f'{df.shape[0]} строка')

201 строка


Заполните средним значением пропуски в столбцах для числовых признаков и самым популярным значением для категориальных признаков
* `num-of-doors`
* `bore`
* `stroke`
* `horsepower`
* `peak-rpm`

In [546]:
df[['num-of-doors', 'bore', 'stroke', 'horsepower', 'peak-rpm']].dtypes

num-of-doors    object
bore            object
stroke          object
horsepower      object
peak-rpm        object
dtype: object

По факту категориальными является признак ```num-of-doors```, а числовыми ```bore```, ```stroke```, ```horsepower```, ```peak-rpm```

In [547]:
#Сделаем замену пропущенных значений записанных как '?'
for i in ['num-of-doors', 'bore', 'stroke', 'horsepower', 'peak-rpm']:
    if i == 'num-of-doors':
        most_frequent_value = df[i].value_counts().idxmax()
        df[i] = df[i].replace('?', most_frequent_value)
    else:
        mean_value = df[i][df[i] != '?'].astype('float64').mean().round(3)
        df[i] = df[i].replace('?', mean_value)
   

## Вопрос для Quiz


Пропуски в столбце `normalized-losses` предскажите при помощи линейной регрессии по признакам
`symboling`, `wheel-base`, `length`, `width`, `height`, `curb-weight`, `engine-size`, `compression-ratio`, `city-mpg`, `highway-mpg` и заполните их предсказаниями

In [548]:
from sklearn.linear_model import LinearRegression

df_train = df.loc[df['normalized-losses'] != '?']
df_test = df.loc[df['normalized-losses'] == '?']

X_train = df_train[['symboling', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-size', 'compression-ratio', 'city-mpg', 'highway-mpg']]
X_test = df_test[['symboling', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-size', 'compression-ratio', 'city-mpg', 'highway-mpg']]
y_train = df_train['normalized-losses']

model = LinearRegression()
model.fit(X_train, y_train)
ypred = model.predict(X_test)

df.loc[df['normalized-losses'] == '?', 'normalized-losses'] = ypred

df.head()


,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,168.072493,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,168.072493,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,134.001799,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


## Вопрос для Quiz

Чему равно предсказание линейной регрессии на первом пропущенном значении? Ответ округлите до целого числа.

In [549]:
ypred[0].round()

168.0

In [550]:
for c in df.columns:
    print(c, len(df[df[c] == '?']))

symboling 0
normalized-losses 0
make 0
fuel-type 0
aspiration 0
num-of-doors 0
body-style 0
drive-wheels 0
engine-location 0
wheel-base 0
length 0
width 0
height 0
curb-weight 0
engine-type 0
num-of-cylinders 0
engine-size 0
fuel-system 0
bore 0
stroke 0
compression-ratio 0
horsepower 0
peak-rpm 0
city-mpg 0
highway-mpg 0
price 0


## 2. Кодирование категориальных признаков

1. Закодируйте бинарные признаки `fuel-type`, `aspiration`, `num-of-doors`, `engine-location` каждый отдельной колонкой, состоящей из 0 и 1.
Единицей кодируйте самую частую категорию.

In [551]:
df[['fuel-type', 'aspiration', 'num-of-doors', 'engine-location']].describe()

,fuel-type,aspiration,num-of-doors,engine-location
count,201,201,201,201
unique,2,2,2,2
top,gas,std,four,front
freq,181,165,115,198


In [552]:
# Кодируем бинарные признаки
for i in ['fuel-type', 'aspiration', 'num-of-doors', 'engine-location']:
    df[i] = df[i].map({df[i].value_counts().idxmax() : 1,
               df[i].value_counts().idxmin() : 0,
               })

2. Вынесите в переменную `y` целевую переменную `price`, а все остальные колонки - в матрицу `X`.

Закодируйте признаки `make`, `body-style`, `engine-type`, `fuel-system` при помощи LeaveOneOutEncoder.

**Дальше все время работайте с объектами `X`, `y`.**

In [553]:
from category_encoders.leave_one_out import LeaveOneOutEncoder

X = df.drop(['price'], axis=1)
y = df.price

encoder = LeaveOneOutEncoder(cols=['make', 'body-style', 'engine-type', 'fuel-system'])

X = encoder.fit_transform(X, y)

## Вопрос для Quiz

Чему равно среднее значение в столбце `body-style` после кодирования? Ответ округлите до целого числа.

In [554]:
X['body-style'].mean().round()

13207.0

3. Закодируйте признак `drive-wheels` при помощи OHE из библиотеки category_encoders.

In [555]:
from category_encoders.one_hot import OneHotEncoder

encoder = OneHotEncoder(cols=['drive-wheels'])
X = encoder.fit_transform(X,y)
#Удалим один из столбцов получившихся поселе OHE кодирования, чтобы избежать мультиколлинеарности
X = X.drop(['drive-wheels_3'], axis=1)

4. В столбце `num-of-cylinders` категории упорядочены по смыслу. Закодируйте их подряд идущими числами, начиная с 1, согласно смыслу.

Подряд идущими числами означает - 1, 2, 3 и так далее без пропусков.

In [556]:
X['num-of-cylinders'] = X['num-of-cylinders'].map({'four': 1, 
                                                    'six': 2, 
                                                    'five': 3, 
                                                    'three': 4, 
                                                    'twelve' : 5, 
                                                    'two' : 6, 
                                                    'eight' : 7})

## Вопрос для Quiz

Сколько столбцов получилось в матрице `X`?

In [557]:
X.shape[1]

26

In [558]:
X['normalized-losses'] = X['normalized-losses'].astype(float)
X['bore'] = X['bore'].astype(float)
X['stroke'] = X['stroke'].astype(float)
X['horsepower'] = X['horsepower'].astype(float)
X['peak-rpm'] = X['peak-rpm'].astype(float)

y = y.astype(float)

Разбейте данные на тренировочную и тестовую часть в пропорции 3 к 1, зафиксируйте random_state = 42.

In [559]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.25, random_state=42)

Масштабируйте данные при помощи MinMaxScaler.

Обучайте масштабирование на тренировочных данных, а потом примените и к трейну, и к тесту.

In [560]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)


Обучите на тренировочных данных линейную регрессию, сделайте предсказание на тесте и вычислите значение $R^2$ на тестовых данных.

Чему равно значение $R^2$ на тестовых данных? Ответ округлите до сотых.

In [561]:
from sklearn.metrics import r2_score, mean_squared_error

model = LinearRegression()
model.fit(X_train, y_train)
ypred = model.predict(X_test)
print(f'R2: {r2_score(y_test, ypred).round(2)}')
print(f'RMSE: {(mean_squared_error(y_test, ypred)**0.5).round(2)}')

R2: 0.91
RMSE: 3058.02


In [562]:
df['price'].median()

10295.0